In [23]:
import ConfigParser
import cea.scripts

In [24]:
CLI_CONFIG = r"C:\Users\darthoma\Documents\GitHub\CityEnergyAnalyst\cea\interfaces\cli\cli.config"

In [32]:
cli_config = ConfigParser.ConfigParser()
cli_config.read(CLI_CONFIG)

['C:\\Users\\darthoma\\Documents\\GitHub\\CityEnergyAnalyst\\cea\\interfaces\\cli\\cli.config']

In [30]:
# make sure each script in cli.config is in the scripts.yml
reload(cea.scripts)
for script, module_path in cli_config.items('scripts'):
    cea_script = cea.scripts.by_name(script)    
    assert cea_script.module == module_path, 'Module paths differ: %s != %s' % (module_path, cea_script.module)

In [37]:
# make sure each script in cli.config has the same parameters as in scripts.yml
reload(cea.scripts)
for script, parameters in cli_config.items('config'):
    cea_script = cea.scripts.by_name(script)
    old_parameters = set(parameters.split())
    new_parameters = set(cea_script.parameters)
    assert old_parameters == new_parameters, 'Parameters for %s differ: %s != %s' % (script, old_parameters, new_parameters)

In [11]:
# make sure the inputs.yml file contains the proper columns
import yaml

import cea.config
import cea.inputlocator
import cea.utilities.dbf

INPUTS_YML = r"C:\Users\darthoma\Documents\GitHub\cea-dashboard\dashboard\inputs\inputs.yml"

config = cea.config.Configuration()
locator = cea.inputlocator.InputLocator(config.scenario)

In [21]:
inputs_yml = yaml.load(open(INPUTS_YML))

for input_name, properties in inputs_yml.items():
    print('processing input: %s' % input_name)
    location = getattr(locator, properties['location'])()
    if location.endswith('.shp'):
        print('skipping shapefile...')
        continue
    print('location: %s' % location)
    dbf = cea.utilities.dbf.dbf_to_dataframe(location)
    dbf_columns = set(dbf.columns)
    yml_columns = set(f['name'] for f in properties['fields'])
    assert dbf_columns == yml_columns, 'Columns differ: %s != %s' % (dbf_columns, yml_columns)
print('done')

processing input: restrictions
location: C:\reference-case-WTP-reduced\WTP_RES_l\inputs\building-properties\restrictions.dbf
processing input: occupancy
location: C:\reference-case-WTP-reduced\WTP_RES_l\inputs\building-properties\occupancy.dbf
processing input: district
skipping shapefile...
processing input: zone
skipping shapefile...
processing input: age
location: C:\reference-case-WTP-reduced\WTP_RES_l\inputs\building-properties\age.dbf
processing input: supply-systems
location: C:\reference-case-WTP-reduced\WTP_RES_l\inputs\building-properties\supply_systems.dbf
processing input: architecture
location: C:\reference-case-WTP-reduced\WTP_RES_l\inputs\building-properties\architecture.dbf
processing input: internal-loads
location: C:\reference-case-WTP-reduced\WTP_RES_l\inputs\building-properties\internal_loads.dbf
done
